In [2]:
import pandas as pd

In [3]:
pd.options.mode.chained_assignment = None

In [3]:
from blocks.get_raw_df import get_raw_df

2024-10-07 11:10:02,461 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-10-07 11:10:02,461 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-10-07 11:10:02,510 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-10-07 11:10:02,510 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-10-07 11:10:02,536 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-10-07 11:10:02,536 INFO sqlalchemy.engine.Engine [raw sql] {}


In [4]:
fill_df = get_raw_df()

2024-10-07 11:10:02,580 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-10-07 11:10:02,584 INFO sqlalchemy.engine.Engine select * from ane_base.parser_app_pricesraw
2024-10-07 11:10:02,586 INFO sqlalchemy.engine.Engine [raw sql] {}


In [5]:
site_links = fill_df[fill_df.date=='2024-10-06'].site_link.unique()

In [6]:
raw_df = fill_df[fill_df.site_link.isin(site_links)]

In [7]:
median_prices = raw_df.groupby(['date', 'category_id'])['price_new'].median()

In [8]:
median_prices = median_prices.reset_index()

In [11]:
raw_df['median_price'] = raw_df.groupby(['date', 'category_id'])['price_new'].transform(lambda x: x.median())

C:\Users\user\AppData\Local\Temp\ipykernel_6524\1819029307.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['median_price'] = raw_df.groupby(['date', 'category_id'])['price_new'].transform(lambda x: x.median())


In [12]:
raw_df['price_ratio'] = raw_df.price_new / raw_df.median_price

C:\Users\user\AppData\Local\Temp\ipykernel_6524\3640978268.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['price_ratio'] = raw_df.price_new / raw_df.median_price


In [13]:
raw_df = raw_df.sort_values('date')

In [14]:
raw_df = raw_df.groupby('site_link').apply(lambda x: x.sort_values('date'))

C:\Users\user\AppData\Local\Temp\ipykernel_6524\3763762248.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  raw_df = raw_df.groupby('site_link').apply(lambda x: x.sort_values('date'))


In [15]:
raw_df = raw_df.reset_index(drop=True)

In [16]:
raw_df['price_change'] = raw_df.groupby('site_link')['price_new'].transform(lambda x: x.pct_change())

In [17]:
raw_df['is_change'] = raw_df['price_change']!=0

In [18]:
raw_df = raw_df.dropna()

In [19]:
sample = raw_df[(raw_df.is_change==True)]['price_ratio']-1

In [20]:
raw_df['is_future_change'] = raw_df.groupby('site_link')['is_change'].transform(lambda x: pd.Series(index=x.index, 
                                                                                                    data=list(x[1:])+[None]))

In [22]:
raw_df[(raw_df.date.between('2019-06-15', '2019-07-15'))&(raw_df.site_title == 'Творог детский Агуша Груша 3.9% 100г')].sort_values('date')

,date,type,category_id,category_title,site_title,price_new,price_old,site_unit,site_link,site_code,miss,id,median_price,price_ratio,price_change,is_change,is_future_change
318001,2019-06-15,food,12,Творог нежирный,Творог детский Агуша Груша 3.9% 100г,28.9,-1,шт,https://www.vprok.ru/product/agusha-agusha-tvo...,vprok,0.0,900621,48.490,0.595999,0.000000,False,False
318002,2019-06-18,food,12,Творог нежирный,Творог детский Агуша Груша 3.9% 100г,28.9,-1,шт,https://www.vprok.ru/product/agusha-agusha-tvo...,vprok,0.0,900622,48.490,0.595999,0.000000,False,True
318003,2019-06-19,food,12,Творог нежирный,Творог детский Агуша Груша 3.9% 100г,26.9,-1,шт,https://www.vprok.ru/product/agusha-agusha-tvo...,vprok,0.0,900623,48.490,0.554754,-0.069204,True,False
318004,2019-06-20,food,12,Творог нежирный,Творог детский Агуша Груша 3.9% 100г,26.9,-1,шт,https://www.vprok.ru/product/agusha-agusha-tvo...,vprok,0.0,900624,48.490,0.554754,0.000000,False,True
318005,2019-06-21,food,12,Творог нежирный,Творог детский Агуша Груша 3.9% 100г,28.9,-1,шт,https://www.vprok.ru/product/agusha-agusha-tvo...,vprok,0.0,900625,48.490,0.595999,0.074349,True,False
318006,2019-06-22,food,12,Творог нежирный,Творог детский Агуша Груша 3.9% 100г,28.9,-1,шт,https://www.vprok.ru/product/agusha-agusha-tvo...,vprok,0.0,900626,48.490,0.595999,0.000000,False,False
318007,2019-06-23,food,12,Творог нежирный,Творог детский Агуша Груша 3.9% 100г,28.9,-1,шт,https://www.vprok.ru/product/agusha-agusha-tvo...,vprok,0.0,900627,48.490,0.595999,0.000000,False,False
318008,2019-06-24,food,12,Творог нежирный,Творог детский Агуша Груша 3.9% 100г,28.9,-1,шт,https://www.vprok.ru/product/agusha-agusha-tvo...,vprok,0.0,900628,48.490,0.595999,0.000000,False,True
318009,2019-06-28,food,12,Творог нежирный,Творог детский Агуша Груша 3.9% 100г,25.9,-1,шт,https://www.vprok.ru/product/agusha-agusha-tvo...,vprok,0.0,900629,48.490,0.534131,-0.103806,True,False
318010,2019-06-29,food,12,Творог нежирный,Творог детский Агуша Груша 3.9% 100г,25.9,-1,шт,https://www.vprok.ru/product/agusha-agusha-tvo...,vprok,0.0,900630,43.900,0.589977,0.000000,False,False


In [23]:
raw_df = raw_df.dropna()

In [24]:
sample = (raw_df[(raw_df.is_future_change==True)].price_ratio-1)#.hist()

In [28]:
raw_df.price_ratio = raw_df.price_ratio-1

C:\Users\user\AppData\Local\Temp\ipykernel_6524\62922306.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df.price_ratio = raw_df.price_ratio-1


In [30]:
sample_new = raw_df[['price_ratio', 'is_future_change']].copy()

In [41]:
sample_new.loc[:, 'interval'] = pd.qcut(sample_new.price_ratio, q=10)